In [58]:
#Install Packages: 
#pip install qiskit

In [31]:
import numpy as np
from math import pi
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
#from qiskit.circuit.library.standard_gates import YGate, RYGate, RXGate, RZGate
from qiskit.converters import circuit_to_dag

In [52]:
#Generating basic circuit example with H, I, Y, S, CNOT, RX, RY, RZ, CZ, x, y, z gates
q = QuantumRegister(3, 'q')
c = ClassicalRegister(3, 'c')
circ = QuantumCircuit(q, c)

circ.rx(pi/3,0)
circ.h(1)
circ.i(0)
circ.y(2)
circ.y(0)
circ.y(1)
circ.s(0)
circ.cnot(1,0)
circ.cnot(1,0)
circ.x(0)
circ.x(1)
circ.z(2)
circ.z(0)
circ.rx(pi/2,1)
circ.ry(pi/3,2)
circ.rx(pi/6,1)
circ.rz(pi/4,0)
circ.cz(1,2)
circ.draw()

┌──────────┐┌───┐   ┌───┐    ┌───┐┌───┐┌───┐┌───┐   ┌───┐    ┌──────────┐»
q_0: ┤ RX(pi/3) ├┤ I ├───┤ Y ├────┤ S ├┤ X ├┤ X ├┤ X ├───┤ Z ├────┤ RZ(pi/4) ├»
     └──┬───┬───┘├───┤   └───┘    └───┘└─┬─┘└─┬─┘├───┤┌──┴───┴───┐├──────────┤»
q_1: ───┤ H ├────┤ Y ├───────────────────■────■──┤ X ├┤ RX(pi/2) ├┤ RX(pi/6) ├»
        ├───┤    ├───┤┌──────────┐               └───┘└──────────┘└──────────┘»
q_2: ───┤ Y ├────┤ Z ├┤ RY(pi/3) ├────────────────────────────────────────────»
        └───┘    └───┘└──────────┘                                            »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«        
«q_0: ───
«        
«q_1: ─■─
«      │ 
«q_2: ─■─
«        
«c: 3/═══
«

In [53]:
#Compiling the circuit - takes in the qubits on which to apply the gate on 
  #and if the gate changes a phase around the axis 
#Returns an equivalent combination of RX, RZ, CZ gates


def compile_to_circuits(circ): 
    
    comp_circ = QuantumCircuit(circ.qregs[0],circ.cregs[0])
    
    for gate in circ:
        gate_name = gate[0].name
        qubits = gate[1]
        cbits = gate[2]
        params = gate[0].params
        
        #
        if gate_name == "cz":
            comp_circ.cz(qubits[0],qubits[1])
            
        elif gate_name == "rx":
            comp_circ.rx(params[0],qubits)

        elif gate_name == "rz":
            comp_circ.rz(params[0],qubits)
       
        #Gates that will be compiled into RX, RZ and CZ Gates: 

        elif gate_name == "ry":
            #RY_theta gate = RX(pi/2), RX(-pi/2), or RZ(theta)
            comp_circ.rx(params[0],qubits)
            comp_circ.rz(-pi/2,qubits)
            comp_circ.rz(pi/2,qubits)

        elif gate_name == "h":
            #H gate = RX(pi/2), RZ(pi/2)
            comp_circ.rx(pi/2,qubits)
            comp_circ.rz(pi/2,qubits)

        elif gate_name == "i":
            #I gate = RX(2pi), RY(2pi), RZ(2pi)
            comp_circ.rx(2*pi,qubits)
            comp_circ.rz(2*pi,qubits)
              
        elif gate_name == "s":
            #S gate = RZ(pi/2)
            comp_circ.rx(pi/2, qubits)  


        elif gate_name == "x":
            #X gate = RX(pi)
            comp_circ.rx(pi, qubits)


        elif gate_name == "y":
            #Y gate = RY(pi), RZ(pi/2), RZ(-pi/2)
            comp_circ.rz(-pi/2, qubits)
            comp_circ.rz(pi/2, qubits)


        elif gate_name == "z":
            comp_circ.rz(pi, qubits)


        elif gate_name == "cx":
            comp_circ.rz(pi/2,qubits[1])
            comp_circ.rx(pi/2,qubits[1])
            comp_circ.cz(qubits[0],qubits[1])
            
    return comp_circ

In [54]:
#Compilation of a basic quantum circuit to a restricted gate set
compiled_circ = compile_to_circuits(circ)
compiled_circ.draw()

┌──────────┐┌───────────┐ ┌──────────┐┌──────────┐ ┌──────────┐»
q_0: ─┤ RX(pi/3) ├┤ RZ(-pi/2) ├─┤ RZ(pi/2) ├┤ RX(pi/2) ├─┤ RZ(pi/2) ├»
      ├──────────┤└┬──────────┤┌┴──────────┤├──────────┤ └──────────┘»
q_1: ─┤ RX(pi/2) ├─┤ RZ(pi/2) ├┤ RZ(-pi/2) ├┤ RZ(pi/2) ├─────────────»
     ┌┴──────────┤ ├──────────┤└─┬────────┬┘├──────────┤┌───────────┐»
q_2: ┤ RZ(-pi/2) ├─┤ RZ(pi/2) ├──┤ RZ(pi) ├─┤ RX(pi/3) ├┤ RZ(-pi/2) ├»
     └───────────┘ └──────────┘  └────────┘ └──────────┘└───────────┘»
c: 3/════════════════════════════════════════════════════════════════»
                                                                     »
«     ┌──────────┐   ┌──────────┐┌──────────┐   ┌────────┐ ┌────────┐ »
«q_0: ┤ RX(pi/2) ├─■─┤ RZ(pi/2) ├┤ RX(pi/2) ├─■─┤ RX(pi) ├─┤ RZ(pi) ├─»
«     └──────────┘ │ └──────────┘└──────────┘ │ ├────────┤┌┴────────┴┐»
«q_1: ─────────────■──────────────────────────■─┤ RX(pi) ├┤ RX(pi/2) ├»
«     ┌──────────┐                              └────────┘└──────────┘»
«q_2: ┤ RZ(pi/2) ├────────────────────────────────────────────────────»
«     └──────────┘                                                    »
«c: 3/════════════════════════════════════════════════════════════════»
«                                                                     »
«     ┌──────────┐   
«q_0: ┤ RZ(pi/4) ├───
«     ├──────────┤   
«q_1: ┤ RX(pi/6) ├─■─
«     └──────────┘ │ 
«q_2: ─────────────■─
«                    
«c: 3/═══════════════
«

In [55]:
#Checking if the original function is equal to the compiled function 
  #with dag and global phase

def dag(M):
    
    M_dag = np.conj(M).T
    return M_dag

def global_phase(M1, M2):
    P = M1.dot(dag(M2))
    I = np.diag(np.ones(M1.shape[0]))
    if P[0][0] == 0:
        equal = False
    else: 
        equal = np.allclose(I,P/P[0][0])
    return equal

In [ ]:
#M_dag, M1, M2: numpy matrices 
#equal: if M1 and M2 are equivalent in global phase

backend = Aer.get_backend('unitary_simulator')

sim = execute(circ, backend)
result = sim.result()
U_circ = result.get_unitary(circ, decimals=3)

sim_compiled = execute(compiled_circ, backend)
result_compiled = sim_compiled.result()
U_compiled_circ = result_compiled.get_unitary(compiled_circ, decimals=3)

print(global_phase(U_circ, U_compiled_circ))

In [57]:
print('Number of gates in original circuit: {0}'.format(circ.size()))
print('Number of gates in compiled circuit: {0}'.format(compiled_circ.size()))

Number of gates in original circuit: 18
Number of gates in compiled circuit: 27
